# 深層学習を用いた言語モデル

*Deep Learning*

深層学習を用いて言語モデルを作成する。  
前章で作成したマルコフモデルの様な「ある単語から次の単語を予測するモデル」をニューラルネットワークを用いて作成する。

本章では、深層学習を活用した言語モデル実装の基礎を学ぶ。

In [1]:
import random
from typing import List

import sentencepiece as spm
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from dlprog import train_progress

In [2]:
prog = train_progress()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')


---

## データセット

wiki40b

In [3]:
textfile = 'data/jawiki.txt'
with open(textfile) as f:
    data = f.readlines()

print('num of data:', len(data))
data[:5] # examples

num of data: 89698


['「教科書には決して載らない」日本人の謎やしきたりを多角的に検証し、日本人のDNAを解明する。新春番組として定期的に放送されており、年末の午前中に再放送されるのが恒例となっている。\n',
 'ライブドア社員であった初代代表取締役社長の山名真由によって企業内起業の形で創業。2005年に株式会社ライブドアから分割されて設立。かつてはライブドアホールディングス（現・LDH）の子会社であったが、ノンコア事業の整理にともない、株式会社ゲオ（現：株式会社ゲオホールディングス）に所有する全株式を譲渡し、同社の完全子会社となった。「ぽすれん」「ゲオ宅配レンタル」のオンラインDVD・CD・コミックレンタルサービス及び「GEO Online」と「ゲオアプリ」のアプリ・ウェブサイト運営の大きく分けて2事業を展開している。以前はDVD販売等のEコマースサービス「ぽすれんストア」、動画配信コンテンツ「ぽすれんBB」や電子書籍配信サービスの「GEO☆Books」事業も行っていた。オンラインDVDレンタル事業では会員数は10万人（2005年9月時点）。2006年5月よりCDレンタルを開始。同業他社には、カルチュア・コンビニエンス・クラブが運営する『TSUTAYA DISCAS』のほか、DMM.comが運営する『DMM.com オンラインDVDレンタル』がある。過去には「Yahoo!レンタルDVD」と「楽天レンタル」の運営を受託していた。\n',
 '2005年の一時期、東京のラジオ局、InterFMで、「堀江社長も使っているライブドアのぽすれん」というキャッチコピーでラジオCMを頻繁に行っていたことがあった。\n',
 '香川県内の農業協同組合の信用事業を統括する県域農協系金融機関であり、県内農業協同組合を会員とする。香川県は全県単一農協の香川県農業協同組合となったが、先に単一農協となった奈良県や沖縄県のケースと異なり、信連の統合は行われなかった。通称は「JA香川信連」または「JAバンク香川」。統一金融機関コードは3037。主に法人顧客を中心としており、個人取引は殆どない。県内の大型商業施設にある、他金融機関管理の共同ATMには香川信連の管轄のものがある。\n',
 '534年（永熙3年）、独孤信の子として生まれた。独孤信が父母妻子を捨てて長安に入ったため、独孤羅は東魏に取り残されて高氏の虜

多すぎるので減らす

In [4]:
data = data[:1000]

textfile = 'data/jawiki_1000.txt'
with open(textfile, 'w') as f:
    for sentence in data:
        f.write(sentence)


---

## サブワード分割

トークン化の手法。

NNでも、マルコフモデル同様、文章をトークン化して学習を行う。  
NNでは文字列をそのまま扱えないので、各トークンにID=クラスラベルを割り当てて、そのシーケンスとして文章を扱う。

前章とは異なり、サブワード分割という新たな手法を使う。ちゃんと理由もある。

1トークンあたりの文字数が多いことは、いくつかのデメリットを生む。  
例えば、トークンの種類が多くなること。深層学習を用いた言語モデルはトークンの種類に比例してパラメータの数が増え、学習が困難になる。  
それから、未知語が増えること。長いトークンは、それだけ多くの情報を持った限定的な言葉ということとなり、これらで語彙が埋まると表現力が落ちる。またデータセットに存在しない言葉を扱うことが困難になる。

サブワード分割は、これらの問題を解決する。  
サブワードと呼ばれる、単語よりも小さな単位に分割することで、1トークンあたりの文字数を減らす。

この手法では、データセットから頻出する文字の並びを学習し、その並びをトークンとして分割する。学習された、トークン化を行うモデルをトークナイザと呼ぶ。  
データセットに合ったトークン化が可能。また語彙数を指定することも可能。指定した語彙数に収まるまで細かく分割してくれる。

ちなみに、ChatGPTが使っているトークナイザはこれ: [OpenAI Platform](https://platform.openai.com/tokenizer)

少し余談。

前章でトークン化について以下のように説明した。

> トークンとはモデルが扱える最小単位のことで、例えば単語が該当する。

ここで、そもそも言語モデルは単語の確率を扱うものなので、絶対にトークン=単語にならないとおかしいとも考えられる。  
実は文章生成においては、単語を最小単位にしなければならない理由はない。極端な話、文字を最小単位として文章を文字の並びと見てもいい訳だ。なんらかのシーケンスとできればそれで充分なのだ。実際、現在有名な言語モデルの多くは単語を最小単位としていない。

ただ、言語モデルは単語の並びに確率を割り当てるモデルだ。単語を最小単位とするモデルだ。  
もしこの定義に厳格になるのであれば、現在有名な多くの言語モデルは言語モデルと呼べないのかもしれんね。しらんけど。

閑話休題。実際にやってみよう。  
sentencepieceというライブラリを用いる。

In [5]:
import sentencepiece as spm

### 学習

語彙数とデータセット（テキストファイル）を指定して、学習させる。

In [ ]:
tokenizer_prefix = 'models/tokenizer_jawiki_1000' # トークナイザのモデル名
vocab_size = 8000 # 語彙数
spm.SentencePieceTrainer.Train(
    input=textfile, # データセット
    model_prefix=tokenizer_prefix,
    vocab_size=vocab_size
)

### トークン化

学習したモデルにテキストを突っ込むとトークン化してくれる。

In [7]:
sp = spm.SentencePieceProcessor(f'{tokenizer_prefix}.model') # モデル読み込み
text = '今日はいい天気だ'
ids = sp.encode(text)
ids

[11, 1993, 6, 2317, 681, 343, 263]

テキストがトークン化され、ID列として取得できる。  
`out_type`を指定すると文字列のリストが取得できる。

In [8]:
sp.encode(text, out_type=str)

['▁', '今日', 'は', 'いい', '天', '気', 'だ']

アンダーバーは空白を意味する。初めにアンダーバーが付くのは仕様。

ID列を文字列に戻すこともできる。

In [9]:
sp.decode(ids)

'今日はいい天気だ'

全てのデータをトークン化（ID化）しよう。  
文章のリスト: `List[str]`を`sp.encode()`に与えるとID列のリスト: `List[List[int]]`が返ってくる。

In [10]:
data_ids = sp.encode(data)
n_vocab = len(sp)
print('num of vocabrary:', n_vocab)
data_ids[0][:10] # example

num of vocabrary: 8000


[11, 18, 6254, 54, 1057, 58, 1685, 79, 122, 17]


---

## 学習データ

NNの学習を行うため、入力と出力のペアを作成する。  
今回はある単語から次に続く単語を予測するモデルを作成するので、ある単語IDとその次の単語IDがペアとなったデータを作成する。

In [11]:
Ids = List[int]
class TextDataset(Dataset):
    def __init__(self, data_ids: List[Ids]):
        self.x = [] # 入力
        self.t = [] # 正解
        for ids in data_ids:
            self.x += ids[:-1]
            self.t += ids[1:]
        self.n_data = len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.t[idx]

    def __len__(self):
        return self.n_data

batch_size = 64
dataset = TextDataset(data_ids)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

n_data = len(dataset)
print('num of data:', n_data)

# examples
x, y = next(iter(dataloader))
print(x)
print(y)

num of data: 243959
tensor([6863,    5,  289, 1301,  100,   21, 1825,   14, 4691, 5159, 2842,   87,
         112,  262,  153,  593,  488,  652, 7911,  522,    3,    7,  496,  640,
        1079,  119,    4, 2066, 2876,  449,    8,    5,  891,  266,   10,  476,
         867, 1507,  999,  370, 1941,    4,   37,   86,   68, 1785,    4, 7985,
        2285,  189,   19,    4, 1748, 2425,    3,    8,  730,  840,  603,  382,
        1678,    7,    4, 5694])
tensor([3395,  644, 3340,   54,  112,   12,   10, 2317, 4939,    7,  592, 1597,
           5,    6,   26,   54,   12,   17,  159,   32, 1664,    4,  256,    3,
          90, 2771, 5458,  316,   12,  101, 4035,  665,   64,   25, 2907,   12,
           4,  142,   12,  566,  205, 4138,    3,   93,    5,   18,  375,   37,
         218,    8,    5,  316,   36,    4,  282,  540, 1789,    8, 6697, 1021,
        1568,  355,  282, 1090])



---

## モデル構築

ある単語IDを入力に取り、次の単語IDを出力するモデルを作成する。

単語IDは入力時にone-hotベクトルに変換する。出力は語彙数分の次元を持つベクトルとする。  
つまりこのタスクは単語の分類問題とも言える。

In [12]:
class LanguageModel(nn.Module):
    def __init__(self, n_vocab: int, hidden_size: int):
        super().__init__()
        self.n_vocab = n_vocab
        self.net = nn.Sequential(
            nn.Linear(n_vocab, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_vocab)
        )

    def forward(self, x):
        """
        x: (batch_size,)
        """
        x = F.one_hot(x, self.n_vocab).to(torch.float32)
            # (batch_size, n_vocab)
        y = self.net(x) # (batch_size, n_vocab)
        return y


---

## 学習

損失関数に交差エントロピーを設定し、通常の分類モデルと同じように学習する。

In [13]:
criterion = nn.CrossEntropyLoss()
def train(model, optimizer, n_epochs, prog_unit=1):
    model.train()
    prog.start(n_iter=len(dataloader), n_epochs=n_epochs, unit=prog_unit)
    for _ in range(1, n_epochs + 1):
        for x, t in dataloader:
            optimizer.zero_grad()
            x = x.to(device) # 入力
            t = t.to(device) # 正解
            y = model(x) # 出力
            loss = criterion(y, t) # 損失
            loss.backward() # 逆伝播
            optimizer.step() # パラメータ更新
            prog.update(loss.item())

In [14]:
hidden_size = 512
model = LanguageModel(n_vocab, hidden_size).to(device)
optimizer = optim.Adam(model.parameters())

In [15]:
train(model, optimizer, n_epochs=20, prog_unit=2)

  1-2/20: ######################################## 100% [00:00:27.19] loss: 6.33308 
  3-4/20: ######################################## 100% [00:00:26.03] loss: 5.00505 
  5-6/20: ######################################## 100% [00:00:26.09] loss: 4.45241 
  7-8/20: ######################################## 100% [00:00:26.87] loss: 4.27896 
 9-10/20: ######################################## 100% [00:00:26.53] loss: 4.21971 
11-12/20: ######################################## 100% [00:00:26.81] loss: 4.18958 
13-14/20: ######################################## 100% [00:00:27.00] loss: 4.17075 
15-16/20: ######################################## 100% [00:00:26.42] loss: 4.15720 
17-18/20: ######################################## 100% [00:00:27.13] loss: 4.14668 
19-20/20: ######################################## 100% [00:00:27.73] loss: 4.13833 



---

## 文章生成

学習したモデルを用いて文章を生成する。

学習させたモデルは、ある単語から次の単語を予測するモデルである。厳密には、ある単語IDを入力に取り、次の単語IDを出力するモデルである。  
さらに厳密に言うと、出力は単語IDではなく確率分布である。この確率分布から次の単語IDをサンプリングすることで、次の単語を生成する。

実際に文章を生成させてみる。ある単語からの次の単語の予測を繰り返すことで文章を生成する。初めの単語だけはこちらで指定する。  
以下の条件を満たすまで単語の生成を続ける。
- 単語数が指定した限界に達する
- 句点が出力される

In [16]:
end_id = sp.piece_to_id('。') # 句点のID

def token_sampling(y: List[float]) -> int:
    """モデルの出力から単語をサンプリングする"""
    probs = F.softmax(y, dim=-1) # 確率分布に変換
    token, = random.choices(range(n_vocab), weights=probs) # サンプリング
    return token

def generate_sentence(
    model: nn.Module,
    start_word: str,
    max_len: int = 30
) -> str:
    model.eval()
    token_id = sp.piece_to_id(start_word)
    token_ids = [token_id]

    # 終了条件を満たすまで単語を生成
    while len(token_ids) <= max_len and token_id != end_id:

        # 入力する単語IDをtensorに変換
        x = torch.tensor(token_id).unsqueeze(0).to(device)

        # 次の単語の確率分布を予測
        y = model(x)[0]
        token_id = token_sampling(y) # サンプリング
        token_ids.append(token_id)

    sentence = sp.decode(token_ids)
    return sentence

In [17]:
start_words = ['今日', '地球', '科学', '人']
for start_word in start_words:
    print(generate_sentence(model, start_word))

今日知られている。
地球とて楽文化大に敗れ準優勝でミッキーの親王、ピエロは1908号を得ることができる。
科学は、有していたが、コンクールである。
人情報番組タイトルの都合上半の間のミュンヘン美術学校、また、KACO、ローマの初期段階のコンスタンティウス1年に亡くなった船に対して、


マルコフモデル同様、直前の単語のみを予測に用いているため、不自然な文章が多く生成される。


---

## 埋め込み層

*Embedding Layer*

NN内部における、単語IDに対する「one-hotベクトル化→線形変換」を行う部分をまとめて**埋め込み層**と表す。  
そもそも、これらの処理は単語IDを受け取って対応するベクトルを出力すること=**単語のベクトル化**と同義である。自然言語処理の世界では単語のベクトル化を**単語の埋め込み**と表現する。

線形変換を行う全結合層は各単語の埋め込み表現（単語ベクトル）を保有していることになる。それらは重みから確認できる。

In [18]:
class Embedding(nn.Module):
    def __init__(self, n_vocab: int, embed_dim: int):
        super().__init__()
        self.n_vocab = n_vocab
        self.fc = nn.Linear(n_vocab, embed_dim, bias=False)

    def forward(self, x):
        x = F.one_hot(x, self.n_vocab).to(torch.float32)
        h = self.fc(x)
        return h

embedding = Embedding(n_vocab=3, embed_dim=5)

x = torch.tensor([0, 1, 2])
h = embedding(x)

print(h, '\n') # 埋め込み層からの出力
print(embedding.fc.weight.T) # 全結合層の重み

tensor([[-0.2905, -0.1799, -0.4095, -0.4943,  0.1511],
        [ 0.1365,  0.4748, -0.2256,  0.0019, -0.0497],
        [ 0.1742,  0.2400,  0.0601,  0.5173, -0.5256]], grad_fn=<MmBackward0>) 

tensor([[-0.2905, -0.1799, -0.4095, -0.4943,  0.1511],
        [ 0.1365,  0.4748, -0.2256,  0.0019, -0.0497],
        [ 0.1742,  0.2400,  0.0601,  0.5173, -0.5256]],
       grad_fn=<PermuteBackward0>)


`PyTorch`には`torch.nn.Embedding`というクラスが実装されているので、それを使うと良い。  
[Embedding — PyTorch 2.0 documentation](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html)

In [19]:
class LanguageModel(nn.Module):
    def __init__(self, n_vocab, hidden_size):
        super().__init__()
        self.net = nn.Sequential(
            nn.Embedding(n_vocab, hidden_size), # Embedding層
            nn.ReLU(),
            nn.Linear(hidden_size, n_vocab)
        )

    def forward(self, x):
        y = self.net(x)
        return y

実際に、言語モデルの学習によって得られる単語ベクトルは良い埋め込み表現として機能することが知られている。単語の埋め込み表現を得ることを目的として言語モデルを学習させることもある。word2vecなどがこれに該当し、これについてはおまけの章で取り上げる。


---

## 特殊トークン

言語モデルを扱う際、データを扱いやすくするために特殊なトークンを考えることがある。  
以下に例を示す。

### *Unknown*

未知語を意味するトークン。  
推論時に学習データに含まれなかった単語に出会ったときに対応できるようになる。学習データ内での出現回数が少ない単語も未知語として扱うことがある。

### *Begin of Sentence*

文章の先頭を意味するトークン。  
先程の例では初めの単語を与える必要があったが、このトークンを作ることで、モデルに「文章の初め」を伝えられるようになる。

### *End of Sentence*

文章の終わりを意味するトークン。  
先程の例では句点が出た場合に生成を止めたが、本来句点は文の終わりであって文章の終わりではない。このトークンを作ることでモデルが文章の終わりを伝えられるようになる。

学習データの全ての文章の該当箇所（BOSであれば文章の初め、EOSであれば文章の終わり）にこれらのトークンを入れてから学習させることで、そのトークンの意味をモデルは理解する。  
またモデルが直接触れるのはトークンではなくトークンIDなので、特殊トークンの名前は何でもいい。他のトークンと重複しないように括弧を付けたりすることが多い。

sentencepieceでは上記3つのトークンがID0, 1, 2にデフォルトで設定されている。

In [20]:
n_vocab = len(sp)
print('ID0:', sp.id_to_piece(0)) # Unknown
print('ID1:', sp.id_to_piece(1)) # Begin of Sentence
print('ID2:', sp.id_to_piece(2)) # End of Sentence

ID0: <unk>
ID1: <s>
ID2: </s>


では、トークン列にBOSとEOSを追加して学習させてみよう。

In [21]:
bos_id = sp.bos_id() # BOSのID
eos_id = sp.eos_id() # EOSのID
for ids in data_ids:
    ids.insert(0, bos_id) # 先頭にBOSを追加
    ids.append(eos_id) # 末尾にEOSを追加

In [22]:
dataset = TextDataset(data_ids)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
model = LanguageModel(n_vocab, 512).to(device)
optimizer = optim.Adam(model.parameters())

In [23]:
train(model, optimizer, n_epochs=10)

 1/10:                                            0% [00:00:00.12] loss: 8.24262 

 1/10: ######################################## 100% [00:00:13.23] loss: 7.01097 
 2/10: ######################################## 100% [00:00:13.06] loss: 5.28967 
 3/10: ######################################## 100% [00:00:12.94] loss: 4.94011 
 4/10: ######################################## 100% [00:00:12.90] loss: 4.79164 
 5/10: ######################################## 100% [00:00:13.05] loss: 4.71006 
 6/10: ######################################## 100% [00:00:13.30] loss: 4.65820 
 7/10: ######################################## 100% [00:00:13.09] loss: 4.61932 
 8/10: ######################################## 100% [00:00:12.93] loss: 4.59368 
 9/10: ######################################## 100% [00:00:12.90] loss: 4.57238 
10/10: ######################################## 100% [00:00:12.94] loss: 4.55414 


では文章を生成する。初めのトークンはBOSとし、EOSが出力されるまで生成を続ける。

In [24]:
unk_id = sp.unk_id() # UNKのID
def token_sampling(y: List[float]) -> int:
    y[unk_id] = -torch.inf # UNKがサンプリングされる確率を0にする
    probs = F.softmax(y, dim=-1)
    token, = random.choices(range(n_vocab), weights=probs)
    return token

def generate_sentence(model, max_len=50):
    model.eval()
    token_id = bos_id
    token_ids = []

    for _ in range(max_len):
        x = torch.tensor(token_id).unsqueeze(0).to(device)
        y = model(x)[0]
        token_id = token_sampling(y)
        if token_id == eos_id: # EOSが生成されたら終了
            break
        token_ids.append(token_id)

    sentence = sp.decode(token_ids)
    return sentence

In [25]:
for _ in range(5):
    print(generate_sentence(model))

次々、県安芸吉田道場に入時だけのデモ・フーヒレンジャク出が作曲に各種手演技を世に晒された際は、歴史的な、過去の文字最も職した。そんな中郷ICを除いた
EGKOblege)化した。20人の被害補償額の称しなが、韓国や、郊の難しい表現したほか、リーグ2連覇を果たし優勝を飾ったことも明日、単根を採択
主翼前後年)の仕方のフェス・チャートでイギリス、10日刊ド Baby Day」と、fr)、亜鉛(Lstimm高山書に送船は掲載誌が、この庁舎や、代わりに
現在の主要イベントの路線に転々とで3560mboard を用いてアウキャロバージアプリマラー「新しいスタイルし、1世はこの戦となり、「ビザンツ帝国の公共建造物から、「ドー講じ
: 遊園設置され、現在は廃止時はこれまでに移動していた。現在は、それらの代表的な表現は積極的に描いている人により先行しやケードラボアレイノ岳ニ長ければるため、流階級を構成する
